<a href="https://colab.research.google.com/github/artilluminati/tgmanager/blob/main/project_farhat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyTelegramBotApi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotApi: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213952 sha256=6482dfa36fe4ebaaf6aef1bfb09615390a060e3b584e3adf4c8e59358e069491
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pyTelegramBotApi


In [ ]:
import numpy as np
import librosa
import soundfile as sf
import io
from six.moves.urllib.request import urlopen
import matplotlib.pyplot as plt
import librosa.display
# import speech_recognition as speech_r

import tensorflow as tf
import os
import time

import telebot
from telebot import types

In [ ]:
# количество эпох обучения нейронки по генерации текста
GENTEXT_EPOCHS = 10

# 3. Генерация текста (Фархат)

In [ ]:
# !wget 'https://github.com/artilluminati/utoolproject/blob/main/%D0%9C%D0%B0%D1%80%D0%B2%D0%B5%D0%BB.txt'

!wget 'https://raw.githubusercontent.com/artilluminati/utoolproject/main/describtion%20of%201000%20film.txt'

--2023-08-13 18:51:30--  https://raw.githubusercontent.com/artilluminati/utoolproject/main/describtion%20of%201000%20film.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1265755 (1.2M) [text/plain]
Saving to: ‘describtion of 1000 film.txt’

describtion of 1000 100%[===================>]   1.21M  --.-KB/s    in 0.06s   

2023-08-13 18:51:31 (21.8 MB/s) - ‘describtion of 1000 film.txt’ saved [1265755/1265755]



In [ ]:
path_to_file = 'describtion of 1000 film.txt'

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print(len(text))

vocab = sorted(set(text))
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab))
tf.strings.unicode_split(text,"UTF-8")
all_ids = ids_from_chars(tf.strings.unicode_split(text,"UTF-8"))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert = True)

seq_length = 100
sequences = ids_dataset.batch(seq_length+1,drop_remainder=True)

def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = (dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))

vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)

    self.dense = tf.keras.layers.Dense(vocab_size)
  def call(self, x, states = None, training = False, return_state = False):
    x = self.embedding(x, training = training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)
    if return_state:
      return x, states
    else:
      return x
model = MyModel(vocab_size = vocab_size, embedding_dim = embedding_dim, rnn_units=rnn_units)

for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',loss=loss, metrics=(['accuracy']))
history = model.fit(dataset, epochs=GENTEXT_EPOCHS)

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    #
    skip_ids = self.ids_from_chars(['UNK'])[:, None]
    sparse_mask = tf.SparseTensor(
        values = [-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())]
    )
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self,inputs,states=None):
    # превращаем строки в айдишники
    input_chars = tf.strings.unicode_split(inputs,"UTF-8")
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # запускаем модель, которую обучали выше
    predicted_logits, states = self.model(x=input_ids, states=states, return_state=True)

    predicted_logits = predicted_logits[:,-1,:]
    predicted_logits = predicted_logits / self.temperature

    # применяем UNK вместо неизвестных символов если они есть
    predicted_logits = predicted_logits + self.prediction_mask

    # генерируем символ
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)

    return predicted_chars, states

one_step_model = OneStep(model, chars_from_ids, ids_from_chars,temperature=0.5)

# states = None
# next_char = tf.constant(['hello'])
# result = [next_char]
# for n in range(1000):
#   next_char, states = one_step_model.generate_one_step(next_char, states=states)
#   result.append(next_char)

# result = tf.strings.join(result)
# print(result[0].numpy().decode('utf-8'))

1264978
Epoch 1/10
195/195 [==============================] - 22s 61ms/step - loss: 2.6283 - accuracy: 0.2894
Epoch 2/10
195/195 [==============================] - 12s 53ms/step - loss: 1.9850 - accuracy: 0.4132
Epoch 3/10
195/195 [==============================] - 12s 52ms/step - loss: 1.6919 - accuracy: 0.5007
Epoch 4/10
195/195 [==============================] - 12s 53ms/step - loss: 1.5064 - accuracy: 0.5529
Epoch 5/10
195/195 [==============================] - 12s 54ms/step - loss: 1.3912 - accuracy: 0.5831
Epoch 6/10
195/195 [==============================] - 13s 56ms/step - loss: 1.3109 - accuracy: 0.6040
Epoch 7/10
195/195 [==============================] - 14s 58ms/step - loss: 1.2486 - accuracy: 0.6209
Epoch 8/10
195/195 [==============================] - 15s 58ms/step - loss: 1.1953 - accuracy: 0.6345
Epoch 9/10
195/195 [==============================] - 13s 56ms/step - loss: 1.1475 - accuracy: 0.6475
Epoch 10/10
195/195 [==============================] - 13s 57ms/step - los

In [ ]:
token = '6462253731:AAE7YJlLzeT9uK3WKuqO5Kv_yvIABGxAeec'

bot = telebot.TeleBot(token)

In [ ]:
@bot.message_handler(commands=(['start', 'generate']))
def generate_text(message):
  mesg = bot.send_message(message.chat.id, 'Напиши какой-нибудь текст, а нейросеть продолжит его за тебя.\nНо пиши его на английском языке!')
  bot.register_next_step_handler(mesg, generate)

def generate(message):
  try:
    bot.send_message(message.chat.id, 'Генерация текста...')
    states = None
    next_char = tf.constant([message.text])
    result = [next_char]
    for n in range(1000):
      next_char, states = one_step_model.generate_one_step(next_char, states=states)
      result.append(next_char)

    result = tf.strings.join(result)
    bot.send_message(message.chat.id, result[0].numpy().decode('utf-8'))
  except Exception as err:
    bot.send_message(message.chat.id, 'Произошла ошибка:', err)

@bot.message_handler(commands=(['return', 'movie', 'movielegacy']))
def generate_text(message):
  try:
    keyboardUrl2 = types.InlineKeyboardMarkup()
    url_button = types.InlineKeyboardButton(text="Вернуться", url="http://t.me/utoolproject_bot")
    keyboardUrl2.add(url_button)
    bot.send_message(message.chat.id, 'Вернуться к основному боту', reply_markup = keyboardUrl2)
  except:
    bot.send_message(message.chat.id, 'Вернуться к основному боту @utoolproject_bot')

In [ ]:
bot.polling()